In [253]:
from vgg16_hybrid_places_1365 import VGG16_Hubrid_1365
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from places_utils import preprocess_input
import numpy as np
import pandas as pd
import os

In [497]:
img_width , img_height = 224, 224
data_size = 3119 
# number of times each image is changed through different data agumentation techniques
# meaning from each images we get 4 images that are altred in some way
data_agum_size = 4

# data paths 
images_path = '/practical_stuff/all_images/'
labels_path = 'practical_stuff/labels.txt'

In [433]:
# load a pre-trained vgg16 model which is trained on images from Places365 database and imagenet data sets
model = VGG16_Hubrid_1365(weights ='places', include_top=False, input_shape = (img_width, img_height, 3))

In [256]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [498]:
# read labels
labels = pd.read_csv(labels_path, sep = ',', header = None, names = ['class','image'])
labels.head()

,class,image
0,other,2378600.jpeg
1,other,2378687.jpeg
2,other,2379071.jpeg
3,other,2379113.jpeg
4,other,2379591.jpeg


In [499]:
print (labels.shape)

(3119, 2)


In [500]:
# define data augmentation 
datagen = ImageDataGenerator(
        rotation_range = 40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest')

In [501]:
# performs data augmentation on images
def data_agumentation(img, data_agum_size):
    img_agumented = np.zeros((data_agum_size, img_width, img_height,3))
    for batch in datagen.flow(img):
        #print ("shape of batches",batch[0].shape)
        img_agumented[i] = preprocess_input(batch[0].reshape((1,batch[0].shape[0],batch[0].shape[1],batch[0].shape[2])))
        i += 1
        if i >= data_agum_size:
            break
    return img_agumented

In [449]:
# returns the class lables for a given row 
def get_target(row):
    label = row['class']
    if label == 'other':
        return 0
    else:
        return 1

In [502]:
# extract features from the 
def extract_features(row, features,targets, augmented = False):
    global counter
    
    if augmented:
        data_agum = np.zeros((img_width, img_height,3))
    
    image_path = os.path.join(os.getcwd() + images_path, row['image'].strip())
    # load an image
    img = image.load_img(image_path,target_size = (img_width, img_height))
    
    # convert the image into np array
    array_image = image.img_to_array(img)
    array_image = np.expand_dims (array_image, axis =0)
    #print (array_image.shape)
    
    if augmented:
        # do data agumentation on this specific image. the resulting images are also pre-processed
        agumented = data_agumentation(array_image,data_agum_size)
   
    # process the image, basically standardization
    array_image = preprocess_input(array_image)
    
    # extract the features by runnung the image array throught the pre-trained vgg16 model
    img_features = model.predict(array_image)
   
   
    # get the class label for this image
    class_label  = get_target(row)
 
    features[counter] = img_features.reshape((7*7*512))
    targets [counter] = class_label
    
    counter +=1
    
    if augmented:
        for img in agumented:
            f_agumented = model.predict(img.reshape(1,img.shape[0], img.shape[1], img.shape[2] ))
            features[counter] = f_agumented.reshape((7*7*512))
            targets[counter] = class_label
            counter +=1
    print (counter)

In [503]:
# to make sure results are reproducible, seed the random number generator
# first shuffle the whoe data set and then 
# split the dataframe into two random samples (80% and 20%) for training and validation.
np.random.seed (100)
mask = np.random.rand(labels.shape[0]) < 0.8
permutation = np.random.permutation(labels.shape[0])

shuffled_dataset = labels.loc[permutation]
train = shuffled_dataset[mask]
valid = shuffled_dataset[~mask]

In [504]:
# place holders for features and the crosponding targets 
# each image has a feature that is 7*7*512 long. 
# Note: 7*7*512 is the size of the last layer of the pre-trained vgg16 model.
# this pre-trained model doesn't have the FC layer
# targets has shape of (3119, 1) as we have 3119 images
data_size = train.shape[0]
features_train = np.zeros ((data_size*data_agum_size + data_size, 7*7*512))
targets_train = np.zeros ((data_size*data_agum_size + data_size, 1))

In [505]:
print (features_train.shape, targets_train.shape)

(12540, 25088) (12540, 1)


In [506]:
# apply data augmentation only on the training part of the data set
# for each row of image call extract_features method
# this call might take some time, but it is a one time process
counter = 0
train.apply(lambda x: extract_features(x, features_train,targets_train, True), axis =1)

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855
860
865
870
875
880
885
890
895
900
905
910
915
920
925
930
935
940
945
950
955
960
965
970
975
980
985
990
995
1000
1005
1010
1015
1020
1025
1030
1035
1040
1045
1050
1055
1060
1065
1070
1075
1080
1085
1090
1095
1100
1105
1110
1115
1120
1125
1130
1135
1140
1145
1150
1155
1160
1165
1170
1175
1180
1185
1190
1195
1200
1205
1210
1215
1220

8420
8425
8430
8435
8440
8445
8450
8455
8460
8465
8470
8475
8480
8485
8490
8495
8500
8505
8510
8515
8520
8525
8530
8535
8540
8545
8550
8555
8560
8565
8570
8575
8580
8585
8590
8595
8600
8605
8610
8615
8620
8625
8630
8635
8640
8645
8650
8655
8660
8665
8670
8675
8680
8685
8690
8695
8700
8705
8710
8715
8720
8725
8730
8735
8740
8745
8750
8755
8760
8765
8770
8775
8780
8785
8790
8795
8800
8805
8810
8815
8820
8825
8830
8835
8840
8845
8850
8855
8860
8865
8870
8875
8880
8885
8890
8895
8900
8905
8910
8915
8920
8925
8930
8935
8940
8945
8950
8955
8960
8965
8970
8975
8980
8985
8990
8995
9000
9005
9010
9015
9020
9025
9030
9035
9040
9045
9050
9055
9060
9065
9070
9075
9080
9085
9090
9095
9100
9105
9110
9115
9120
9125
9130
9135
9140
9145
9150
9155
9160
9165
9170
9175
9180
9185
9190
9195
9200
9205
9210
9215
9220
9225
9230
9235
9240
9245
9250
9255
9260
9265
9270
9275
9280
9285
9290
9295
9300
9305
9310
9315
9320
9325
9330
9335
9340
9345
9350
9355
9360
9365
9370
9375
9380
9385
9390
9395
9400
9405
9410
9415


1902    None
1468    None
2937    None
1764    None
522     None
2380    None
623     None
2699    None
1616    None
1093    None
2324    None
277     None
310     None
1801    None
6       None
2997    None
531     None
718     None
1648    None
185     None
2815    None
3105    None
2483    None
1906    None
2594    None
3101    None
1851    None
173     None
2711    None
2118    None
        ... 
95      None
2316    None
2576    None
1423    None
2573    None
2492    None
2696    None
2251    None
1384    None
1280    None
2774    None
970     None
952     None
1001    None
2611    None
1498    None
854     None
3052    None
250     None
1455    None
301     None
252     None
9       None
2082    None
1778    None
2950    None
2636    None
1038    None
1566    None
1159    None
Length: 2508, dtype: object

In [507]:
data_size = valid.shape[0]
features_valid = np.zeros ((data_size*data_agum_size + data_size, 7*7*512))
targets_valid = np.zeros ((data_size*data_agum_size + data_size, 1))

In [508]:
print (features_valid.shape, targets_valid.shape)

(3055, 25088) (3055, 1)


In [509]:
# do not apply data augmentation on the validation part of the data set
counter = 0
valid.apply(lambda x: extract_features(x, features_valid,targets_valid), axis =1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


2721    None
124     None
2476    None
2076    None
773     None
2970    None
3047    None
1849    None
2602    None
3020    None
116     None
2645    None
2106    None
1033    None
2021    None
2620    None
1131    None
467     None
1171    None
597     None
1867    None
74      None
1450    None
1432    None
2318    None
1471    None
1229    None
55      None
1195    None
110     None
        ... 
223     None
2572    None
976     None
747     None
814     None
2080    None
2714    None
2843    None
481     None
1986    None
1724    None
1698    None
1684    None
2854    None
2618    None
1101    None
2917    None
2043    None
1707    None
2366    None
3088    None
2617    None
2303    None
2111    None
278     None
1641    None
1610    None
2361    None
2433    None
275     None
Length: 611, dtype: object

In [510]:
# saving features and labels for further training
# a classifier will be trained on these features and labels
np.save('features_train', features_train)
np.save('targets_train', targets_train)

In [511]:
# saving features and labels for further training
# a classifier will be trained these features and their crossponding labels
np.save('features_valid', features_valid)
np.save('targets_valid', targets_valid)